In [6]:
import sys, os
sys.path.append(os.path.abspath("../src"))

from nets import MLP
import torch
import torch.nn as nn
import numpy as np

In [7]:
net = MLP(1, 2, 3, 20, activation_fn = nn.Tanh())

In [8]:
net(torch.tensor(np.random.rand(10, 1), dtype=torch.float32))

tensor([[-0.0170,  0.0354],
        [-0.0010,  0.0445],
        [-0.0228,  0.0325],
        [-0.0290,  0.0298],
        [-0.0102,  0.0391],
        [-0.0042,  0.0426],
        [-0.0241,  0.0319],
        [-0.0198,  0.0340],
        [-0.0227,  0.0326],
        [-0.0117,  0.0382]], grad_fn=<AddmmBackward0>)